In [1]:
#!curl -o /tmp/roads.geojson https://raw.githubusercontent.com/tpolong/geodocker/master/roads.geojson 

In [2]:
import geopyspark as gps
import numpy as np
import fiona
import pyproj
from pyspark import SparkContext
from functools import partial
from shapely.geometry import Point, MultiPolygon, LineString, box,MultiLineString,shape
from shapely.ops import transform
from geonotebook.wrappers import VectorData, TMSRasterData

In [3]:
conf = gps.geopyspark_conf(master="local[*]", appName="map-algebra")
pysc = SparkContext(conf=conf)

In [4]:
with fiona.open("/tmp/roads.geojson") as source:
    roads_crs = source.crs['init']
    roads = [MultiLineString(shape(line['geometry'])) for line in source]

In [10]:
def create_partial_reprojection_func(crs):
    return partial(pyproj.transform,
                   pyproj.Proj(init=crs),
                   pyproj.Proj(init='epsg:3857'))
reprojected_roads = [transform(create_partial_reprojection_func(roads_crs), road) for road in roads]

In [12]:
#raster_poly_1 = box(0.0, 0.0, 5.0, 10.0)
#raster_poly_2 = box(3.0, 6.0, 15.0, 20.0)
#raster_poly_3 = box(13.5, 17.0, 30.0, 20.0)

#raster_multi_poly = MultiPolygon([raster_poly_1, raster_poly_2, raster_poly_3])
rasterize_options = gps.RasterizerOptions(includePartial=True, sampleType='PixelIsArea')

road_raster = gps.rasterize(geoms=reprojected_roads,
                            crs="EPSG:3857",
                            zoom=14,
                            fill_value=1,
                            cell_type=gps.CellType.FLOAT32,
                            options=rasterize_options,
                            num_partitions=50)

In [13]:
weighted_layer=road_raster 
weight=weighted_layer.pyramid()
weighted_histogram = weight.get_histogram()
weighted_color_map = gps.ColorMap.build(breaks=weighted_histogram,
                                        colors='viridis')
tms = gps.TMS.build(source=weight,
                    display=weighted_color_map)
M.add_layer(TMSRasterData(tms), name="Weighted Layer")

In [9]:
M.set_center(-122.45, 37.75, 11)